In [1]:
import mlflow
import os
import requests
import pandas as pd
from minio import Minio
from io import BytesIO

# ======================
# LightGBM 학습 (불균형 보정 + 조기종료) - MLflow 연동
# ======================
# 테스트용 데이터 임의 생성 (입력)
import numpy as np
import pandas as pd
import os
import pickle
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import socket

In [2]:
mlflow_server_ip = socket.gethostbyname("mlflow-server")
mlflow.set_tracking_uri(f"http://{mlflow_server_ip}:5000")
mlflow.set_experiment("test_classifier")

<Experiment: artifact_location='s3://mlflow/3', creation_time=1761710958662, experiment_id='3', last_update_time=1761710958662, lifecycle_stage='active', name='test_classifier', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [3]:
# MinIO 클라이언트 초기화
client = Minio(
    "mlflow-minio:9000",  # MinIO 서버 주소
    access_key="minio",
    secret_key="minio123",
    secure=False  # HTTPS를 사용하지 않는 경우 (프로덕션 환경에서는 True로 설정)
)

In [4]:
# 모든 년도 파일 리스트
bucket_name = "prepro"
years = ['2020', '2021', '2022', '2023', '2024', '2025']
dfs = []

for year in years:
    object_name = f"restaurant_{year}.csv"
    response = client.get_object(bucket_name, object_name)
    data_stream = BytesIO(response.read())
    df_temp = pd.read_csv(data_stream)
    dfs.append(df_temp)
    response.close()
    response.release_conn()

# 모든 DataFrame 합치기
df = pd.concat(dfs, ignore_index=True)

In [5]:
df[:2]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2020-12-31,운영중,영업/정상,경상북도 구미시 임은1길 3,황금육꼬기,한식,NaN,NaN,경상북도,구미시,임은1길,NaN,3,1
1,일반음식점,2020-12-31,운영중,영업/정상,충청북도 청주시 흥덕구 오송읍 만수1길 11,푸드박스 오송점,김밥(도시락),NaN,NaN,충청북도,청주시,흥덕구,오송읍 만수1길,11,1


In [6]:
df = df.copy()
df = df.drop(columns=['num'])

In [7]:
len(df)

61015

In [8]:
df[:2]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지
0,일반음식점,2020-12-31,운영중,영업/정상,경상북도 구미시 임은1길 3,황금육꼬기,한식,NaN,NaN,경상북도,구미시,임은1길,NaN,3
1,일반음식점,2020-12-31,운영중,영업/정상,충청북도 청주시 흥덕구 오송읍 만수1길 11,푸드박스 오송점,김밥(도시락),NaN,NaN,충청북도,청주시,흥덕구,오송읍 만수1길,11


In [9]:
le = LabelEncoder()
label_data = le.fit_transform(df['영업상태명'])

In [10]:
df['label'] = label_data

In [11]:
df[:2]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,label
0,일반음식점,2020-12-31,운영중,영업/정상,경상북도 구미시 임은1길 3,황금육꼬기,한식,NaN,NaN,경상북도,구미시,임은1길,NaN,3,0
1,일반음식점,2020-12-31,운영중,영업/정상,충청북도 청주시 흥덕구 오송읍 만수1길 11,푸드박스 오송점,김밥(도시락),NaN,NaN,충청북도,청주시,흥덕구,오송읍 만수1길,11,0


In [12]:
# 주소지 컬럼 인코딩
import re

# 번지 숫자 추출 함수
def extract_lot_number(lot_str):
    """번지에서 숫자를 추출합니다 (예: '3', '11', '169-3' -> 3, 11, 169)"""
    if pd.isna(lot_str) or lot_str == '' or lot_str == 'nan':
        return 0
    # 첫 번째 숫자 추출
    match = re.search(r'\d+', str(lot_str))
    return int(match.group()) if match else 0

# 번지 컬럼 숫자로 변환
df['번지_숫자'] = df['번지'].apply(extract_lot_number)

# 시도, 시군구, 읍면동 LabelEncoder 인코딩
cols = ['시도', '시군구', '읍면동', '구분']
address_encoders = {}

for col in cols:
    if col in df.columns:
        le = LabelEncoder()
        # NaN 값을 '기타'로 채우기
        df[col] = df[col].fillna('기타')
        df[f'{col}_encoded'] = le.fit_transform(df[col])
        address_encoders[col] = le

# 도로명은 카디널리티가 너무 높아 제외하거나, 빈도 기반으로 필터링
# 필요시 아래 코드 활성화
# from collections import Counter
# road_freq = Counter(df['도로명'].fillna(''))
# popular_roads = {road for road, count in road_freq.items() if count > 10}
# df['도로명_인기'] = df['도로명'].apply(lambda x: x if x in popular_roads else '기타')
# le_road = LabelEncoder()
# df['도로명_인기_encoded'] = le_road.fit_transform(df['도로명_인기'])

print("주소지 인코딩 완료")

주소지 인코딩 완료


In [13]:
df[:2]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,label,번지_숫자,시도_encoded,시군구_encoded,읍면동_encoded,구분_encoded
0,일반음식점,2020-12-31,운영중,영업/정상,경상북도 구미시 임은1길 3,황금육꼬기,한식,NaN,NaN,경상북도,구미시,임은1길,NaN,3,0,3,3,32,10870,21
1,일반음식점,2020-12-31,운영중,영업/정상,충청북도 청주시 흥덕구 오송읍 만수1길 11,푸드박스 오송점,김밥(도시락),NaN,NaN,충청북도,청주시,흥덕구,오송읍 만수1길,11,0,11,17,190,14085,3


In [14]:
# 사용할 feature 선정
unused_cols = ['개방서비스명', '인허가일자', '폐업일자', '영업상태명', '소재지', 
               'label', '시도', '시군구', '읍면동', '번지', '도로명', 
               '시설명', '남성종사자수', '여성종사자수']  # 시설명, 종사자수도 제외
features = [col for col in df.columns if col not in unused_cols]

X = df[features]
y = df['label']

print(f"Feature 개수: {len(features)}")
print(f"Feature 목록: {features}")

Feature 개수: 6
Feature 목록: ['구분', '번지_숫자', '시도_encoded', '시군구_encoded', '읍면동_encoded', '구분_encoded']


In [15]:
features

['구분', '번지_숫자', '시도_encoded', '시군구_encoded', '읍면동_encoded', '구분_encoded']

In [16]:
X

,구분,번지_숫자,시도_encoded,시군구_encoded,읍면동_encoded,구분_encoded
0,한식,3,3,32,10870,21
1,김밥(도시락),11,17,190,14085,3
2,기타,365,1,112,6489,2
3,기타,10,6,55,699,2
4,경양식,7,2,116,4878,1
...,...,...,...,...,...,...
61010,기타,21,6,61,14003,2
61011,한식,60,2,73,6264,21
61012,한식,2055,9,65,2041,21
61013,호프/통닭,71,3,32,6552,22


In [17]:
# 사용할 feature 선정: 비식별화 및 미래예측에 직접적 영향을 주지 않는 열만 사용 (예시)
# 주의: Cell 13에서 이미 정의되었으므로 사용하지 않음 (중복 실행 방지)
# unused_cols = ['개방서비스명', '인허가일자', '폐업일자', '영업상태명', '소재지', 'label']
# features = [col for col in df.columns if col not in unused_cols]

# X = df[features]
# y = df['label']


In [18]:
print("X 컬럼:", X.columns.tolist())
print("X 타입:", X.dtypes)
print("X 샘플:")
print(X[:2])
print('='*100)
print(f'y : {set(y)}')

X 컬럼: ['구분', '번지_숫자', '시도_encoded', '시군구_encoded', '읍면동_encoded', '구분_encoded']
X 타입: 구분             object
번지_숫자           int64
시도_encoded      int64
시군구_encoded     int64
읍면동_encoded     int64
구분_encoded      int64
dtype: object
X 샘플:
        구분  번지_숫자  시도_encoded  시군구_encoded  읍면동_encoded  구분_encoded
0       한식      3           3           32        10870          21
1  김밥(도시락)     11          17          190        14085           3
y : {0, 1}


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [20]:
RANDOM_STATE = 123
ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

In [21]:
!pip install boto3

In [22]:
import os

# MinIO/S3 접근을 위한 AWS credentials 설정
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'  # .env.train 파일의 값으로 변경
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'  # .env.train 파일의 값으로 변경
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://mlflow-minio:9000'
os.environ['AWS_ENDPOINT_URL'] = 'http://mlflow-minio:9000'

In [23]:
# train 데이터 형상 및 통계 확인
print("Train features shape:", X_train.shape)
print("Test features shape:", X_test.shape)
print("Train target value distribution:\n", y_train.value_counts())
print("Test target value distribution:\n", y_test.value_counts())
print("\nTrain feature preview:\n", X_train.head())

# categorical_feature 자동 지정: 범주형 특성 리스트 추출
# 예시: 범주형 변수명을 리스트로 명시 (자동 인코딩된 경우?)
cat_features = []
for col in X_train.columns:
    if "encoded" in col or X_train[col].dtype == "object":
        cat_features.append(col)

print("Categorical features for LGBM:", cat_features)

Train features shape: (48812, 6)
Test features shape: (12203, 6)
Train target value distribution:
 label
0    34537
1    14275
Name: count, dtype: int64
Test target value distribution:
 label
0    8780
1    3423
Name: count, dtype: int64

Train feature preview:
              구분  번지_숫자  시도_encoded  시군구_encoded  읍면동_encoded  구분_encoded
60672        한식     11           3          136        13054          21
25345  식육(숯불구이)     92           1           21         1128          10
57467        한식     15           9            1         3253          21
6503         일식     59           8           38         1749          12
13935        한식     20           9            1          264          21
Categorical features for LGBM: ['구분', '시도_encoded', '시군구_encoded', '읍면동_encoded', '구분_encoded']


In [ ]:
with mlflow.start_run():
    # 정수 컬럼 경고 해결: 전부 float64로 변환
    # 문자열 컬럼이 있다면 float로 변환 가능한 컬럼만 선택
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    X_train = X_train[numeric_cols].astype('float64')
    X_test = X_test[numeric_cols].astype('float64')
    
    # cat_features도 업데이트 (X_train에 존재하는 컬럼만)
    cat_features_updated = [col for col in cat_features if col in numeric_cols]
    
    # 파라미터 로깅
    mlflow.log_params({
        "n_estimators": 1500,
        "learning_rate": 0.04,
        "num_leaves": 63,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_lambda": 1.0,
        "class_weight": "balanced",
        "test_size": 0.2,
        "random_state": 123,
        "categorical_features": cat_features_updated
    })
    
    lgbm = LGBMClassifier(
        n_estimators=1500,
        learning_rate=0.04,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        class_weight="balanced",
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    # 학습 메트릭을 수집하기 위한 callback
    evals_result = {}
    lgbm.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric="auc",
        callbacks=[
            lgb.early_stopping(stopping_rounds=80, verbose=True),
            lgb.record_evaluation(evals_result)
        ],
        categorical_feature=cat_features_updated if cat_features_updated else "auto"
    )
    
    # 학습 과정 메트릭 로깅 (시간 경과 시각화용)
    for metric_name, metric_values in evals_result.get('valid_0', {}).items():
        for epoch, value in enumerate(metric_values):
            mlflow.log_metric(f"train_{metric_name}", value, step=epoch)
    
    # 모델 저장 (로컬 파일)
    model_filename = f"lgbm_classifier_run_{mlflow.active_run().info.run_id}.pkl"  # 동적 이름
    with open(os.path.join(ARTIFACT_DIR, model_filename), "wb") as f:
        pickle.dump(lgbm, f)
    
    # 평가 메트릭을 MLflow로 로깅
    proba = lgbm.predict_proba(X_test)[:, 1]
    pred_default = (proba >= 0.5).astype(int)
    
    roc = roc_auc_score(y_test, proba)
    precision = average_precision_score(y_test, proba)
    
    mlflow.log_metrics({
        "roc": roc,
        "precision": precision,
        "f1_score": f1_score(y_test, pred_default),
        "accuracy": (y_test == pred_default).mean(),
        "best_iteration": lgbm.best_iteration_,
        "num_features": len(X_train.columns)
    })
    
    # 모델 로깅 (MLflow에 모델도 저장, 개선된 방식: signature와 input_example 포함)
    from mlflow.models import infer_signature

    # 1. 모델이 예상하는 입력 예제 생성 (float64 유지)
    input_example = X_train.iloc[:5]

    # 2. 모델의 예측 결과로 시그니처 추론
    signature = mlflow.models.infer_signature(
        X_train.iloc[:5],
        lgbm.predict(X_train.iloc[:5])
    )

    # 3. 시그니처와 예제를 포함하여 모델 저장
    mlflow.lightgbm.log_model(
        lgbm, 
        name="lgbm_classifier",  # ← 여기서 모델 이름 변경 가능
        signature=signature,
        input_example=input_example
    )
    
    print(f"\n[MLflow] 실험 저장 완료!")

In [28]:
# with mlflow.start_run():
#     # 정수 컬럼 경고 해결: 전부 float64로 변환
#     # 문자열 컬럼이 있다면 float로 변환 가능한 컬럼만 선택
#     numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
#     X_train = X_train[numeric_cols].astype('float64')
#     X_test = X_test[numeric_cols].astype('float64')
    
#     # cat_features도 업데이트 (X_train에 존재하는 컬럼만)
#     cat_features_updated = [col for col in cat_features if col in numeric_cols]
    
#     # 파라미터 로깅
#     mlflow.log_params({
#         "n_estimators": 1500,
#         "learning_rate": 0.04,
#         "num_leaves": 63,
#         "subsample": 0.8,
#         "colsample_bytree": 0.8,
#         "reg_lambda": 1.0,
#         "class_weight": "balanced",
#         "test_size": 0.2,
#         "random_state": 123,
#         "categorical_features": cat_features_updated
#     })
    
#     lgbm = LGBMClassifier(
#         n_estimators=1500,
#         learning_rate=0.04,
#         num_leaves=63,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         reg_lambda=1.0,
#         class_weight="balanced",
#         random_state=RANDOM_STATE,
#         n_jobs=-1
#     )

#     # 학습 메트릭을 수집하기 위한 callback
#     evals_result = {}
#     lgbm.fit(
#         X_train, y_train,
#         eval_set=[(X_test, y_test)],
#         eval_metric="auc",
#         callbacks=[
#             lgb.early_stopping(stopping_rounds=80, verbose=True),
#             lgb.record_evaluation(evals_result)
#         ],
#         categorical_feature=cat_features_updated if cat_features_updated else "auto"
#     )
    
#     # 학습 과정 메트릭 로깅 (시간 경과 시각화용)
#     for metric_name, metric_values in evals_result.get('valid_0', {}).items():
#         for epoch, value in enumerate(metric_values):
#             mlflow.log_metric(f"train_{metric_name}", value, step=epoch)
    
#     # 모델 저장 (로컬 파일)
#     model_filename = f"lgbm_classifier_run_{mlflow.active_run().info.run_id}.pkl"  # 동적 이름
#     with open(os.path.join(ARTIFACT_DIR, model_filename), "wb") as f:
#         pickle.dump(lgbm, f)
    
#     # 평가 메트릭을 MLflow로 로깅
#     proba = lgbm.predict_proba(X_test)[:, 1]
#     pred_default = (proba >= 0.5).astype(int)
    
#     roc = roc_auc_score(y_test, proba)
#     precision = average_precision_score(y_test, proba)
    
#     mlflow.log_metrics({
#         "roc": roc,
#         "precision": precision,
#         "f1_score": f1_score(y_test, pred_default),
#         "accuracy": (y_test == pred_default).mean(),
#         "best_iteration": lgbm.best_iteration_,
#         "num_features": len(X_train.columns)
#     })
    
#     # 모델 로깅 (MLflow에 모델도 저장, 개선된 방식: signature와 input_example 포함)
#     from mlflow.models import infer_signature

#     # 1. 모델이 예상하는 입력 예제 생성 (float64 유지)
#     input_example = X_train.iloc[:5]

#     # 2. 모델의 예측 결과로 시그니처 추론
#     signature = mlflow.models.infer_signature(
#         X_train.iloc[:5],
#         lgbm.predict(X_train.iloc[:5])
#     )

#     # 3. 시그니처와 예제를 포함하여 모델 저장
#     mlflow.lightgbm.log_model(
#         lgbm, 
#         name="lgbm_classifier",  # ← 여기서 모델 이름 변경 가능
#         signature=signature,
#         input_example=input_example
#     )
    
#     print(f"\n[MLflow] 실험 저장 완료!")
#     model_uri = mlflow.get_artifact_uri("lgbm_classifier")

#     ############
#     registered_model = mlflow.register_model(
#         model_uri, 
#         "lgbm_classifier"
#     )

#  # 방법 1: runs:/ URI 직접 사용 (권장)
#     import bentoml

#     run_id = mlflow.active_run().info.run_id
#     model_uri = f"runs:/{run_id}/lgbm_classifier"  # ← 명시적 경로

#     bento_model = bentoml.mlflow.import_model(
#         'lgbm_classifier',
#         model_uri=model_uri,  # ← runs URI 사용
#         labels={
#             "team": "bento",
#             "stage": "dev",
#             "accuracy": (y_test == pred_default).mean(),
#         }
#     )
#     # bentoml.models.push(bento_model.tag)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 14275, number of negative: 34537
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001750 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4024
[LightGBM] [Info] Number of data points in the train set: 48812, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 80 rounds
Early stopping, best iteration is:
[248]	valid_0's auc: 0.639281	valid_0's binary_logloss: 0.640358


Registered model 'lgbm_classifier' already exists. Creating a new version of this model...
2025/10/29 07:05:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lgbm_classifier, version 3
Created version '3' of model 'lgbm_classifier'.
'labels' should be a dict[str, str] and enforced by BentoML. Converting all values to string.



[MLflow] 실험 저장 완료!
🏃 View run agreeable-squid-136 at: http://172.18.0.4:5000/#/experiments/3/runs/c40d288f419644688d32293b01e960ac
🧪 View experiment at: http://172.18.0.4:5000/#/experiments/3
